## Zuger Handelsregister

## Imports

In [4]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"

# 1. Scrapen der Daten

## 1.1 Homepage öffnen, Suchfeld ansteuern

In [ ]:
#Wir starten den Browser auf
driver = webdriver.Chrome(executable_path='C:\chromedriver.exe') #bei Windows ist das anders
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
time.sleep(3)
#Hier drücke ich den Cookie-Banner weg
driver.find_element_by_id("j_idt198").click() #ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
#Ich öffne das zusätzliche Panel, ohne dass man dies auf der Seite sieht
su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
driver.execute_script('arguments[0].click();', su_fel_ad)
time.sleep(2)
# Den Filter ansteuern, wo ich Domiziladresse auwählen kann
filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
filt.send_keys('Domiziladresse')
filt.send_keys(Keys.ENTER)
time.sleep(2)
# Hier klappe ich die Zusätzlichen Suchfelder auf
driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
time.sleep(2)

## 1.2 Erste Seite abrufen
hier nun zuerst alle erste Seiten - so dass ich überall eine Anzahl habe. 
Danach gehe ich nur noch jene durch, die Firmen auf den Seiten haben.

In [ ]:
dff=pd.read_csv(folderpath+"/BRFrames/alle_Adressen")

In [ ]:
dff

In [ ]:
listeadress=[]
counter=0
for adress in tqdm(dff["Adresse"]): #die Adressen durchgehen
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.send_keys(adress) #Adresse Eingeben
    search_adress.send_keys(Keys.ENTER) #Enter drücken
    time.sleep(8)
    page = driver.page_source.encode("utf-8") #Page in Variable speichern
    
    with open ("HRSeiten_neu/frontpage"+str(counter)+".htm", "wb+") as file:
        file.write(page)
        file.close()
        time.sleep(1)
    pagename = "frontpage"+str(counter)+".htm"        
    counter=counter+1 
        
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.clear()
    time.sleep(1)
   
    
    minidictadress={"Adresse":adress, "Dokument":pagename}
    listeadress.append(minidictadress)

In [ ]:
dfl=pd.DataFrame(listeadress)

In [6]:
dfl.to_csv(r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg\BRFrames\adresse_file")

NameError: name 'dfl' is not defined

# 2. Nun Infos roh rausziehen
Roheinträge aus den gespeicherten Files abfragen
Danach mit Split in Pandas Infos rausziehen

In [16]:
anzahl_liste=[]
path=folderpath+"/HRSeiten/"
for page in tqdm(os.listdir(path)): #Ich erstelle direkt aus dem Folder eine Liste.
    file=open(path+page, 'r')
    page=page
    text=file.read()
    soup=BeautifulSoup(text, 'html.parser')
    div_list=soup.find_all("div")
    if len(div_list) == 97: #Wenn es keine Firma hat, hat es nur 97 Listen-Punkte
        anzahl= str("0") # Ich vermerke, dass es keine Firmen hat
        input_list=div_list[50].find_all("input") # Hier der Adressen-Eintrag
        adresse_o=input_list[0]
        adresse=str(adresse_o) #HIer noch ein Trick, dass er mir die Sache nicht als Liste ausspuckt
       
    else:
        anzahl= div_list[-26].text #Angabe mit Anzahl Firmen auslesen.
        input_list_f=div_list[-51].find_all("input") 
        adresse_f=input_list_f[0]
        adresse=str(adresse_f) # hier derselbe Trick wie oben
        

    minidict_a={"Seite":page,"Anzahl":anzahl, "Adresse":adresse}
    anzahl_liste.append(minidict_a)

100%|██████████| 5650/5650 [03:16<00:00, 28.82it/s]


In [17]:
# Hier nun die rohe Liste mit den Adressen, Files und Anzahl Firmen.
df_adan=pd.DataFrame(anzahl_liste)

In [18]:
df_adan

,Seite,Anzahl,Adresse
0,frontpage0.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
1,frontpage1.htm,\n\t\t\t\tNo. of companies found: 5\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
2,frontpage10.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
3,frontpage100.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
4,frontpage1000.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
...,...,...,...
5645,frontpage995.htm,\n\t\t\t\tNo. of companies found: 18\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
5646,frontpage996.htm,\n\t\t\t\tNo. of companies found: 90\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
5647,frontpage997.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
5648,frontpage998.htm,\n\t\t\t\tNo. of companies found: 10\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."


In [23]:
pd.set_option('display.max_colwidth', -1)

In [26]:
df_adan.to_csv(folderpath+"/BRFrames/Stadtzug_frontpages_ausgelesen_roh")# abspeichern

In [3]:
#df_adan=pd.read_csv("BRFrames/Stadtzug_frontpages_ausgelesen_roh")

## 2.1 Zahl und Adresse extrahieren
mit Regex vom Rest trennen

In [4]:
df_adan.shape #Ausganglage waren 4690. Offenbar habe ich einige doppelt. egal.

(5650, 4)

In [5]:
# Hier nun die Formel, um aus allen die Zahl rausziehen zu können
df_adan["Anzahl"]=df_adan["Anzahl"].str.extract("(\d+)")

In [6]:
# und noch die Adresse
df_adan["Adresse"]=df_adan["Adresse"].str.extract("value=\"(.*)\"")

In [ ]:
df_adan.to_csv(folderpath+"/BRFrames/StadtZug_extrahiert_unbereinigt")

## 2.2 Umlaut-Codes bereinigen
wichtig für den Abgleich mit der Adress-Liste. 
- So sehe ich, ob Adressen noch fehlen
- Kann so später die Dataframes wieder zusammenfügen für Geopandas

In [8]:
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¼","ü")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¶","ö")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¤","ä")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã„","Ä")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã–","Ö")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ãœ","Ü")

In [9]:
df_adan.drop_duplicates("Seite", inplace=True)

In [10]:
df_adan.shape

(5650, 4)

# 3. Abgleichen von Adress-Frame und Scrape-Frame
Sind alle Adressen runtergezogen 

In [11]:
df_st_zug=pd.read_csv(folderpath+"/BRFrames/adresse_Zug") # die ursprünglichen Adressen der Stadt Zug

In [12]:
df_st_zug.shape

(4690, 15)

In [13]:
# so zusammenführen dass auch die fehlenden Daten sichtbar werden
df_abgleich=pd.merge(df_adan, df_st_zug, on="Adresse", how="outer") 

In [14]:
df_abgleich

,Unnamed: 0_x,Seite,Anzahl,Adresse,Unnamed: 0_y,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
0,0,frontpage0.htm,0,"Aabachstrasse 1, Zug",3464,190988543,0,ZG,1711,Zug,Aabachstrasse,1,6300,0,Zug,2681243.278,1225319.466,de
1,1,frontpage1.htm,5,"Aabachstrasse 10, Zug",3369,190564448,0,ZG,1711,Zug,Aabachstrasse,10,6300,0,Zug,2681328.366,1225274.400,de
2,2,frontpage10.htm,0,"Aabachstrasse 2, Zug",1430,321148,0,ZG,1711,Zug,Aabachstrasse,2,6300,0,Zug,2681294.530,1225154.193,de
3,3,frontpage100.htm,0,"Alte Baarerstrasse 4, Zug",1203,320895,0,ZG,1711,Zug,Alte Baarerstrasse,4,6300,0,Zug,2682278.228,1225122.918,de
4,4,frontpage1000.htm,0,"Chamerstrasse 180.2, Zug",4077,191775911,0,ZG,1711,Zug,Chamerstrasse,180.2,6300,0,Zug,2678903.909,1226520.613,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5645,5645,frontpage995.htm,18,"Chamerstrasse 175, Zug",2928,160009603,0,ZG,1711,Zug,Chamerstrasse,175,6300,0,Zug,2679236.637,1226168.313,de
5646,5646,frontpage996.htm,90,"Chamerstrasse 176, Zug",2957,160012752,0,ZG,1711,Zug,Chamerstrasse,176,6300,0,Zug,2679346.948,1226153.547,de
5647,5647,frontpage997.htm,0,"Chamerstrasse 177, Zug",2919,160007699,0,ZG,1711,Zug,Chamerstrasse,177,6300,0,Zug,2679221.076,1226153.141,de
5648,5648,frontpage998.htm,10,"Chamerstrasse 18, Zug",1433,321151,0,ZG,1711,Zug,Chamerstrasse,18,6300,0,Zug,2681295.335,1225130.509,de


In [15]:
df_abgleich.drop_duplicates("Adresse", inplace=True)# Doppelte Einträge löschen

In [19]:
df_abgleich.shape, df_st_zug.shape #Kontrollieren ob es gleich viele sind wie bei der Stadt Zug

((4690, 18), (4690, 15))

## 3.1 Wenn die Länge noch nicht stimmt und es noch "NaN"-Werte hat

#### Alle Adressen filtern, die NAN-Werte haben - also nicht funktionierten https://stackoverflow.com/questions/22551403/python-pandas-filtering-out-nan-from-a-data-selection-of-a-column-of-strings

In [18]:
df_abgleich[df_abgleich["Seite"].notnull()==False] 

,Unnamed: 0_x,Seite,Anzahl,Adresse,Unnamed: 0_y,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP


In [ ]:
df_abgleich.to_csv(folderpath+"BRFrames/Stadt_Zug_fehlende_adressem")

## 3.2 Wenns stimmt, alle Adressen da sind und das Frame bereinigt mit Ä und Ü:
### Abspeichern

In [ ]:
df_abgleich.to_csv(folderpath+"/BRFrames/Stadt_Zug_sauber")

In [ ]:
df_sauber=df_abgleich

# 4. Daten anschauen, sortieren

**Fragen**
- Wo hat es am meisten Firmen?
- wieviele Adressen haben mehr als 20 Firmen (und müssten noch "nachgescrapet" werden)?

In [5]:
df_sauber=pd.read_csv(folderpath+"/BRFrames/Stadt_Zug_sauber")

## 4.1 Klein machen und Zahlen in int umwandeln

In [13]:
# Ich brauche nur einen Teil
df_saukl=df_sauber[["Adresse","Anzahl","Seite","STRNAME"]]
df_saukl

,Adresse,Anzahl,Seite,STRNAME
0,"Aabachstrasse 1, Zug",0,frontpage0.htm,Aabachstrasse
1,"Aabachstrasse 10, Zug",5,frontpage1.htm,Aabachstrasse
2,"Aabachstrasse 2, Zug",0,frontpage10.htm,Aabachstrasse
3,"Alte Baarerstrasse 4, Zug",0,frontpage100.htm,Alte Baarerstrasse
4,"Chamerstrasse 180.2, Zug",0,frontpage1000.htm,Chamerstrasse
...,...,...,...,...
4685,"Chamerstrasse 175, Zug",18,frontpage995.htm,Chamerstrasse
4686,"Chamerstrasse 176, Zug",90,frontpage996.htm,Chamerstrasse
4687,"Chamerstrasse 177, Zug",0,frontpage997.htm,Chamerstrasse
4688,"Chamerstrasse 18, Zug",10,frontpage998.htm,Chamerstrasse


In [16]:
df_saukl["Anzahl"]=df_saukl["Anzahl"].astype("int")

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_saukl.dtypes

Adresse    object
Anzahl      int32
Seite      object
STRNAME    object
dtype: object

## 4.2 Sortieren  und anschauen

In [29]:
# Hier die Topadressen. Wie vermutet grösstenteils an der Baarerstrasse.
df_saukl.sort_values("Anzahl",ascending=False).head(40)

,Adresse,Anzahl,Seite,STRNAME
4219,"Bahnhofstrasse 21, Zug",328,frontpage590.htm,Bahnhofstrasse
4173,"Baarerstrasse 75, Zug",277,frontpage548.htm,Baarerstrasse
4682,"Chamerstrasse 172, Zug",220,frontpage992.htm,Chamerstrasse
157,"Dammstrasse 19, Zug",219,frontpage1145.htm,Dammstrasse
1435,"Industriestrasse 47, Zug",197,frontpage2312.htm,Industriestrasse
4205,"Bahnhofstrasse 10, Zug",193,frontpage577.htm,Bahnhofstrasse
4113,"Baarerstrasse 137, Zug",161,frontpage494.htm,Baarerstrasse
3779,"Baarerstrasse 10, Zug",156,frontpage446.htm,Baarerstrasse
4058,"Bahnhofplatz, Zug",155,frontpage4711.htm,Bahnhofplatz
4078,"Baarerstrasse 112, Zug",152,frontpage473.htm,Baarerstrasse


In [10]:
# Anzahl der Adressen, die nochmals durchgegangen werden müssten - wenn ich die UIDs abfragen will
df_saukl[df_saukl["Anzahl"]>20]

,Adresse,Anzahl,Seite
16,"Chamerstrasse 30, Zug",39,frontpage1011.htm
44,"Chamerstrasse 56, Zug",27,frontpage1037.htm
97,"Chamerstrasse 77, Zug",31,frontpage1087.htm
98,"Chamerstrasse 79, Zug",30,frontpage1088.htm
146,"Chollerstrasse 35, Zug",36,frontpage1135.htm
...,...,...,...
4574,"Bundesplatz 16, Zug",35,frontpage863.htm
4645,"Chamerstrasse 115, Zug",35,frontpage956.htm
4666,"Chamerstrasse 14, Zug",46,frontpage978.htm
4682,"Chamerstrasse 172, Zug",220,frontpage992.htm


In [11]:
# Alle Adressen mit mindesrens einem Eintrag
df_saukl[df_saukl["Anzahl"]>0]

,Adresse,Anzahl,Seite
1,"Aabachstrasse 10, Zug",5,frontpage1.htm
6,"Chamerstrasse 22, Zug",1,frontpage1002.htm
7,"Chamerstrasse 24, Zug",4,frontpage1003.htm
9,"Chamerstrasse 26, Zug",3,frontpage1005.htm
10,"Chamerstrasse 28, Zug",5,frontpage1006.htm
...,...,...,...
4683,"Chamerstrasse 173, Zug",1,frontpage993.htm
4684,"Chamerstrasse 174, Zug",19,frontpage994.htm
4685,"Chamerstrasse 175, Zug",18,frontpage995.htm
4686,"Chamerstrasse 176, Zug",90,frontpage996.htm


In [27]:
# Hotspots sind also tatsächlich die Baarersrasse, Bahnhofstrasse, Und weiter hinten ann Industriestrasse sowie Chamerstrasse 
df_saukl[df_saukl["Anzahl"]>20].groupby("STRNAME").count().sort_values("Anzahl", ascending=False)

,Adresse,Anzahl,Seite
STRNAME,,,
Baarerstrasse,30,30,30
Bahnhofstrasse,12,12,12
Industriestrasse,8,8,8
Chamerstrasse,8,8,8
Poststrasse,5,5,5
Gubelstrasse,5,5,5
Alpenstrasse,4,4,4
Gotthardstrasse,4,4,4
Grienbachstrasse,3,3,3


In [36]:
df_saukl[df_saukl["Adresse"].str.contains("Unter Altstadt")==True]

,Adresse,Anzahl,Seite,STRNAME
3246,"Unter Altstadt 1, Zug",0,frontpage3975.htm,Unter Altstadt
3247,"Unter Altstadt 10, Zug",38,frontpage3976.htm,Unter Altstadt
3248,"Unter Altstadt 10.1, Zug",0,frontpage3977.htm,Unter Altstadt
3249,"Unter Altstadt 11, Zug",6,frontpage3978.htm,Unter Altstadt
3250,"Unter Altstadt 12, Zug",5,frontpage3979.htm,Unter Altstadt
3252,"Unter Altstadt 13, Zug",1,frontpage3980.htm,Unter Altstadt
3253,"Unter Altstadt 13.1, Zug",0,frontpage3981.htm,Unter Altstadt
3254,"Unter Altstadt 14, Zug",0,frontpage3982.htm,Unter Altstadt
3255,"Unter Altstadt 15, Zug",19,frontpage3983.htm,Unter Altstadt
3256,"Unter Altstadt 16, Zug",0,frontpage3984.htm,Unter Altstadt
